In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [3]:
data = pd.concat([
       pd.read_csv("../input/pageviews/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("../input/pageviews_complemento/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
])
data

/home/jloscalzo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,CONTENT_CATEGORY,CONTENT_CATEGORY_BOTTOM,CONTENT_CATEGORY_TOP,FEC_EVENT,ON_SITE_SEARCH_TERM,PAGE,SITE_ID,USER_ID
0,1,1,1,2018-03-30 07:35:48,1,1,1,0
1,2,2,2,2018-03-30 07:35:52,1,2,2,0
2,2,2,2,2018-03-30 07:36:11,1,3,3,0
3,2,2,2,2018-03-30 07:36:16,1,4,3,0
4,2,2,2,2018-03-30 07:41:38,1,5,2,0
...,...,...,...,...,...,...,...,...
4933415,2,2,2,2018-10-23 08:52:46,1,164,3,10935
4933416,2,2,2,2018-10-23 08:52:54,1,165,3,10935
4933417,2,2,2,2018-10-23 08:53:18,1,210,3,10935
4933418,2,2,2,2018-10-23 08:53:21,1,211,3,10935


In [4]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [5]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_BOTTOM
haciendo CONTENT_CATEGORY_TOP
haciendo ON_SITE_SEARCH_TERM
haciendo PAGE
haciendo SITE_ID


In [6]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [8]:
y_prev = pd.read_csv("../input/conversiones/conversiones.csv")
y_prev.head()

,mes,anio,USER_ID
0,7.0,2018.0,1410.0
1,8.0,2018.0,10755.0
2,8.0,2018.0,8270.0
3,10.0,2018.0,7558.0
4,9.0,2018.0,10731.0


In [9]:
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(
        set(X_train.index))
y_train.loc[list(idx)] = 1

In [10]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score

fi = []
test_probs = []
i = 0
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    learner = LGBMClassifier(n_estimators=10000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)])
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
fi = pd.concat(fi, axis=1).mean(axis=1)

In [12]:
test_probs.to_csv("benchmark.csv", header=True)

In [8]:
os.listdir("./")

['__notebook__.ipynb', '__output__.json', 'benchmark.zip']